# Multiprocessing and Multithreading in Python: A Comparative Study
**Multiprocessing** and **Multithreading** are both mechanisms to achieve concurrency & parallelism in execution of programs and utilization of system resources. However, depending on the specific requiremets of the application, there are some differences in the way they achieve this which should be kept in mind while making a choice.
1. Multiprocessing:
    - Multiprocessing involves creating multiple processes, each with its own Python interpreter and memory space.
    - Processes run independently and can execute in parallel on different CPU cores.
    - Processes communicate through inter-process communication (IPC) mechanisms like pipes, queues, or shared memory.
    - Each process has its own Global Interpreter Lock (GIL), allowing for true parallel execution and utilization of multiple CPU cores.
    - Multiprocessing is suitable for CPU-bound tasks that can benefit from parallel execution, as well as for achieving better resource isolation and handling crashes or exceptions in separate processes.  

2. Multithreading:
    - Multithreading involves creating multiple threads within a single process, sharing the same memory space and resources.
    - Threads run concurrently, but due to the Global Interpreter Lock (GIL) in CPython, only one thread can execute Python bytecode at a time.
    - Multithreading is more suitable for I/O-bound tasks or situations where the blocking of one thread does not significantly impact the overall performance.
    - Threads can communicate and coordinate by sharing data through shared memory, but proper synchronization mechanisms (such as locks or semaphores) are required to avoid data races and ensure thread safety.
    - Although multithreading may not fully utilize multiple CPU cores for CPU-bound tasks, it can still provide benefits in scenarios where parallel execution is not the primary concern.  

In python commonly used module for Multiprocessing is **multiprocessing** and **threading** for multithreading. Let's try out some use cases with them.  


## Common code for imports and functions definitions
We will create two functions- one CPU bound and another to simulate I/O bound when CPU is mostly idle. 
In the following snippet cpu_bound and io_bound functions have been defined. We will be creating driver codes for various cases.


In [28]:
import os
import time
from multiprocessing import Process, current_process
from threading import Thread, current_thread, active_count

NUM = 240000000
TIME = 10

def cpu_bound(num):
    pid, process_name, thread_name = get_context()
    print(f"| {pid} | {process_name} | {thread_name} | ---> Countdown BEGIN")
    
    start = time.time()
    while num > 0:
        num -= 1
    end = time.time()

    print(f"| {pid} | {process_name} | {thread_name} | ---> Countdown END after {end - start} seconds")
    
def io_bound(sec):
    pid, process_name, thread_name = get_context()
    print(f"| {pid} | {process_name} | {thread_name} | ---> Waiting BEGIN")
    
    start = time.time()
    time.sleep(sec)
    end = time.time()

    print(f"| {pid} | {process_name} | {thread_name} | ---> Waiting END after {end - start} seconds")
    
def get_context():
    return (os.getpid(), current_process().name, current_thread().name)


## I/O bound execution without concurrency
In each driver codes we will run one of the functions twice.

In [15]:
if __name__=="__main__":
    start = time.time()
    io_bound(TIME)
    io_bound(TIME)
    end = time.time()
    print(f"Finished in {end - start} seconds")

| 42 | MainProcess | MainThread | ---> Waiting BEGIN
| 42 | MainProcess | MainThread | ---> Waiting END after 9.999000072479248 seconds
| 42 | MainProcess | MainThread | ---> Waiting BEGIN
| 42 | MainProcess | MainThread | ---> Waiting END after 10.0 seconds
Finished in 20.00499987602234 seconds


Both the function calls are executed in tandom and take around 10s each. Only MainProcess and MainThread were used.

## I/O bound execution with multithreading

In [5]:
if __name__=="__main__":
    start = time.time()
    thread1 = Thread(target = io_bound, args = [TIME])
    thread2 = Thread(target = io_bound, args = [TIME])
    thread1.start()
    thread2.start()
    thread1.join()
    thread2.join()
    end = time.time()
    print(f"Finished in {end - start} seconds")

| 92746 | MainProcess | Thread-7 (io_bound) | ---> Waiting BEGIN
| 92746 | MainProcess | Thread-8 (io_bound) | ---> Waiting BEGIN
| 92746 | MainProcess | Thread-7 (io_bound) | ---> Waiting END after 10.009523630142212 seconds
| 92746 | MainProcess | Thread-8 (io_bound) | ---> Waiting END after 10.009634733200073 seconds
Finished in 10.012132167816162 seconds


Thread-7 and Thread-8 were started at the same time within MainProcess. Both threads calling the I/O bound function took around 10s each to finish. However the whole process completed within around 10s only. That means second thread utilized CPU/processor when it was idle after calling sleep function. Both threads were executed concurrently, rather parallely to be more specific.

## I/O bound execution with multiprocessing

In [4]:
if __name__=="__main__":
    start = time.time()
    process1 = Process(target = io_bound, args = [TIME])
    process2 = Process(target = io_bound, args = [TIME])
    process1.start()
    process2.start()
    process1.join()
    process2.join()
    end = time.time()
    print(f"Finished in {end - start} seconds")

| 6368 | Process-3 | MainThread | ---> Waiting BEGIN
| 6370 | Process-4 | MainThread | ---> Waiting BEGIN
| 6368 | Process-3 | MainThread | ---> Waiting END after 10.010463237762451 seconds
| 6370 | Process-4 | MainThread | ---> Waiting END after 10.009718418121338 seconds
Finished in 10.062548160552979 seconds


Results with multiprocessing are as good as with multithreading here. However for I/O bound tasks multiprocessing should be a second option as creating processes itself is a cpu intesive task and takes more time than creating threads. Processes require more resources and need special mechanism like IPC to communicate among themselves. On the other hand threads share same memory space and communicate easily among themselves.

## CPU bound execution without concurrency

In [31]:
if __name__=="__main__":
    start = time.time()
    cpu_bound(NUM)
    cpu_bound(NUM)
    end = time.time()
    print(f"Finished in {end - start} seconds")

| 4827 | MainProcess | MainThread | ---> Countdown BEGIN
| 4827 | MainProcess | MainThread | ---> Countdown END after 10.136963844299316 seconds
| 4827 | MainProcess | MainThread | ---> Countdown BEGIN
| 4827 | MainProcess | MainThread | ---> Countdown END after 10.366931676864624 seconds
Finished in 20.504101753234863 seconds


Both functions take around 10 seconds each and the total time taken by the process seems to the a sum of the two as both executions happened in tandem.

## CPU bound execution with multithreading

In [30]:
if __name__=="__main__":
    start = time.time()
    thread1 = Thread(target = cpu_bound, args = [NUM])
    thread2 = Thread(target = cpu_bound, args = [NUM])
    thread1.start()
    thread2.start()
    thread1.join()
    thread2.join()
    end = time.time()
    print(f"Finished in {end - start} seconds")

| 4827 | MainProcess | Thread-13 (cpu_bound) | ---> Countdown BEGIN
| 4827 | MainProcess | Thread-14 (cpu_bound) | ---> Countdown BEGIN
| 4827 | MainProcess | Thread-13 (cpu_bound) | ---> Countdown END after 18.83767008781433 seconds
| 4827 | MainProcess | Thread-14 (cpu_bound) | ---> Countdown END after 18.856546878814697 seconds
Finished in 18.88337516784668 seconds


Though both the threads seem to stat and finish at around same time, there isn't any significant improvement in time in comparison to the previous case when the function is called twice one after the other. We can say thay the tasks' execution happened concurrently but it's not parallel in true sense. Global Interpreter Lock (GIL) prevents second thread to use CPU till the first threads finishes its comutational task. Running with different values of NUM reveals that the minor differen in overall time is or constant order.

## CPU bound execution with multiprocessing

In [29]:
if __name__=="__main__":
    start = time.time()
    process1 = Process(target = cpu_bound, args = [NUM])
    process2 = Process(target = cpu_bound, args = [NUM])
    process1.start()
    process2.start()
    process1.join()
    process2.join()
    end = time.time()
    print(f"Finished in {end - start} seconds")

| 7270 | Process-7 | MainThread | ---> Countdown BEGIN
| 7272 | Process-8 | MainThread | ---> Countdown BEGIN
| 7270 | Process-7 | MainThread | ---> Countdown END after 11.069042921066284 seconds
| 7272 | Process-8 | MainThread | ---> Countdown END after 11.318077802658081 seconds
Finished in 11.368677616119385 seconds


This time it seems to be parallel execution in true sense with only sight overhand in time to finish which makes sense as processes take that time to get created. Main process created two subprocesses, 7270 & 7272, which do the countdown on separate CPUs/processors parallely.

## Summary

CPU intestive tasks should be executed using multiprocessing to utilize the aailable compute resources fully and achieve parallelism in its true sense. Though non CPU bound tasks (like I/O bound) perform well with multithreading as well as multiprocessing, multithreading should be the first choice to avoid overhang time, make use of easly inter-thread communication and to spare compute resources to other tasks in need.